In [2]:
from sentence_transformers import SentenceTransformer, util
from tqdm import tqdm
import torch
import json
from tqdm import tqdm
import re
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()
import nltk
from nltk.tokenize import sent_tokenize

# Download the Punkt Tokenizer Models (only need to do this once)
nltk.download('punkt')

[nltk_data] Downloading package punkt to /home/21fl11/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [3]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "3"

In [4]:
import json
# with open("duplicate_triplet_dataset_evaluation_V2.json","r") as f:
#     eval_data = json.load(f)
with open('stack_overflow_python_java.json', 'r') as f:
    combined_data = json.load(f)

In [5]:
combined_data

{'123': {'question': 'Java lib or app to convert CSV to XML file?',
  'context': '<p>Is there an existing application or library in <em>Java</em> which will allow me to convert a <code>CSV</code> data file to <code>XML</code> file?  </p>\n\n<p>The <code>XML</code> tags would be provided through possibly the first row containing column headings.</p>\n',
  'accepted_answer_id': '183',
  'tags': '<java><xml><csv><data-conversion>',
  'accepted_answer': '<p>Maybe this might help: <a href="http://jsefa.sourceforge.net/quick-tutorial.html" rel="noreferrer">JSefa</a></p>\n\n<p>You can read CSV file with this tool and serialize it to XML.</p>\n'},
 '126': {'question': 'How would you access Object properties from within an object method?',
  'context': '<p>What is the "purist" or "correct" way to access an object\'s properties from within an object method that is not a getter/setter method?</p>\n\n<p>I know that from outside of the object you should use a getter/setter, but from within would yo

In [6]:
import random

# my_dict = {'a': 1, 'b': 2, 'c': 3, 'd': 4}
n = 50  # Number of items to sample
sampled_pairs = random.sample(list(combined_data.items()), n)


In [7]:
sampled_pairs

[('29149098',
  {'question': 'JavaFX ContextMenu how do I get the clicked Object?',
   'context': '<p>I am learning javafx.scene.control.ContextMenu, and right now I am facing a problem:</p>\n\n<p>how do I get the clicked Object from EventHandler? both event.source() and event.target() return the MenuItem.</p>\n\n<p>let me explain with an example:\nwhat should I write inside the function handle?</p>\n\n<pre><code>    TextField text = new TextField();\n    Label label1 = new Label("hello");\n    Label label2 = new Label("world");\n    Label label3 = new Label("java");\n\n    ContextMenu menu = new ContextMenu();\n    MenuItem item = new MenuItem("copy to text field");\n    menu.getItems().add(item);\n    item.setOnAction(new EventHandler(){\n        public void handle(Event event) {\n            //I want to copy the text of the Label I clicked to TextField\n            event.consume();\n        }\n    });\n\n    label1.setContextMenu(menu);\n    label2.setContextMenu(menu);\n    label3.

In [ ]:
with open('sampled_dict_RQ1.json', 'w') as json_file:
    json.dump(sampled_pairs, json_file)

In [5]:
# Function to filter the data
def filter_data(data):
    filtered_data = []
    for key, value in data.items():
        # Check if 'accepted_answer' exists
        if 'accepted_answer' in value:
            # Remove 'accepted_answer_id' attribute
            value.pop('accepted_answer_id', None)
            filtered_data.append(value)
    return filtered_data

In [6]:
cleaned_combined_data = filter_data(combined_data)

In [7]:
# len(cleaned_combined_data),len(combined_data)

In [8]:
tag_list = [item['tags'] for item in cleaned_combined_data]

In [10]:
len(tag_list)

3428217

In [11]:
def convert_tags(tags_list):
    converted_list = []
    for tags in tags_list:
        # Removing leading '<' and trailing '>', then split by '><'
        individual_tags = tags.strip('<>').split('><')
        converted_list.append(individual_tags)
    return converted_list

# Convert the tags
converted_tags = convert_tags(tag_list)

In [12]:
converted_tags


[['java', 'xml', 'csv', 'data-conversion'],
 ['java', 'php', 'oop', 'theory'],
 ['python', 'xml'],
 ['java', 'generics', 'warnings', 'casting', 'type-safety'],
 ['python', 'macos', 'fonts', 'photoshop'],
 ['python', 'windows', 'image', 'pdf'],
 ['python', 'continuous-integration', 'extreme-programming'],
 ['c#', 'java', 'integer', 'int'],
 ['python', 'sql', 'database', 'oracle', 'cx-oracle'],
 ['python', 'arrays', 'iteration'],
 ['python', 'django', 'view', 'oop'],
 ['python', 'mysql', 'postgresql', 'bpgsql'],
 ['python', 'python-itertools'],
 ['javascript', 'html', 'css', 'fonts'],
 ['python', 'oop', 'methods', 'monkeypatching'],
 ['python', 'performance', 'data-structures', 'graph-theory'],
 ['asp.net', 'javascript', 'validation'],
 ['python', 'syntax', 'binary', 'integer', 'literals'],
 ['python', 'xml', 'database', 'protocol-buffers'],
 ['python'],
 ['python', 'operating-system', 'cross-platform', 'platform-agnostic'],
 ['asp.net', 'javascript', 'anchor', 'quotes'],
 ['python', 'li

In [13]:
import pickle

# Saving the converted_tags list to a file
with open('converted_tags.pkl', 'wb') as file:
    pickle.dump(converted_tags, file)


In [9]:
tag_list

['<java><xml><csv><data-conversion>',
 '<java><php><oop><theory>',
 '<python><xml>',
 '<java><generics><warnings><casting><type-safety>',
 '<python><macos><fonts><photoshop>',
 '<python><windows><image><pdf>',
 '<python><continuous-integration><extreme-programming>',
 '<c#><java><integer><int>',
 '<python><sql><database><oracle><cx-oracle>',
 '<python><arrays><iteration>',
 '<python><django><view><oop>',
 '<python><mysql><postgresql><bpgsql>',
 '<python><python-itertools>',
 '<javascript><html><css><fonts>',
 '<python><oop><methods><monkeypatching>',
 '<python><performance><data-structures><graph-theory>',
 '<asp.net><javascript><validation>',
 '<python><syntax><binary><integer><literals>',
 '<python><xml><database><protocol-buffers>',
 '<python>',
 '<python><operating-system><cross-platform><platform-agnostic>',
 '<asp.net><javascript><anchor><quotes>',
 '<python><list><tuples>',
 '<java><swing><jbutton>',
 '<php><python><ftp><webserver><ftplib>',
 '<javascript><html><popup>',
 '<pyth

In [3]:
import re
from nltk.tokenize import sent_tokenize

def clean_sentence(sen):
    sen = sen.replace('\n', ' ')

    # Remove all tags except <p> and </p>
    sen = re.sub(r'(?i)<(?!\/?p\b)[^>]*>', '', sen)

    sen = re.sub(r'\s+', ' ', sen).strip()
    return sen

def split_and_clean_p_tags(sentences):
    new_sentences = []
    for sen in sentences:
        # Remove <p> and </p> tags
        cleaned_sentence = re.sub(r'</?p>', '', sen)
        cleaned_sentence = cleaned_sentence.strip()  # Remove leading/trailing whitespace

        if cleaned_sentence:  # Check if the sentence is not empty
            new_sentences.append(cleaned_sentence)

    return new_sentences
def custom_sent_tokenize(text):
    text = text.replace('&quot;', '"')

    sections = re.findall(r'(?:<pre><code>.*?</code></pre>|<li><p>.*?</li>|<ol><p>.*?</ol>|<ul><p>.*?</ul>|<menu><p>.*?</menu>|<p>.*?</p>)', text, flags=re.DOTALL)

    output_sentences = []
    code_snippets = []
    urls = []
    url_positions = []
    code_positions = []
    # print(sections)
    position_counter = 0
    for section in sections:
        if '<pre><code>' in section:
            match = re.search(r'<pre><code>(.*?)</code></pre>', section, flags=re.DOTALL)
            if match:
                code = match.group(1)
                code_snippets.append(code)
                code_positions.append(position_counter)
            else:
                code = None  # or handle the absence of a match as needed
            # position_counter += 1
        elif '<li>' in section:
            list_items = re.findall(r'<li>(.*?)</li>', section, flags=re.DOTALL)
            list_sentences = [clean_sentence(item) for item in list_items]
            output_sentences.extend(list_sentences)
            extracted_urls = re.findall(r'<a href="(.*?)"', section)
            
            if len(extracted_urls)>0:
                for u in extracted_urls:
                    urls.append(u)
                    url_positions.append(position_counter)  
            if len(list_sentences)>0:
                position_counter += 1
        elif '<ul>' in section:  
            list_items = re.findall(r'<ul>(.*?)</ul>', section, flags=re.DOTALL)
            list_sentences = [clean_sentence(item) for item in list_items]
            output_sentences.extend(list_sentences)
            extracted_urls = re.findall(r'<a href="(.*?)"', section)
            if len(extracted_urls)>0:
                for u in extracted_urls:
                    urls.append(u)
                    url_positions.append(position_counter)
            
            if len(list_sentences)>0:
                position_counter += 1
        
        elif '<ol>' in section:  
            list_items = re.findall(r'<ol>(.*?)</ol>', section, flags=re.DOTALL)
            list_sentences = [clean_sentence(item) for item in list_items]
            output_sentences.extend(list_sentences)
            extracted_urls = re.findall(r'<a href="(.*?)"', section)
            if len(extracted_urls)>0:
                for u in extracted_urls:
                    urls.append(u)
                    url_positions.append(position_counter)
            
            if len(list_sentences)>0:
                position_counter += 1  

        elif '<menu>' in section:  
            list_items = re.findall(r'<menu>(.*?)</menu>', section, flags=re.DOTALL)
            list_sentences = [clean_sentence(item) for item in list_items]
            output_sentences.extend(list_sentences)
            extracted_urls = re.findall(r'<a href="(.*?)"', section)
            
            if len(extracted_urls)>0:
                for u in extracted_urls:
                    urls.append(u)
                    url_positions.append(position_counter)
            
            if len(list_sentences)>0:
                position_counter += 1 

        elif '<p>' in section:
            paragraph = re.search(r'<p>(.*?)</p>', section, flags=re.DOTALL).group(1)
            sentences = sent_tokenize(paragraph)
            cleaned_sentences = [clean_sentence(sen) for sen in sentences]
            output_sentences.extend(cleaned_sentences)

            for cs in range(len(cleaned_sentences)):
                # extracted_urls = re.findall(r'<a href="(.*?)".*?>', cs)
                extracted_urls = re.findall(r'<a href="(.*?)"', sentences[cs], flags=re.DOTALL)
                if len(extracted_urls)>0:
                    for u in extracted_urls:
                        urls.append(u)
                        url_positions.append(position_counter)
                        
                position_counter += 1
           
    final_output = split_and_clean_p_tags(output_sentences)

    knowledge_dictionary = []
    for i in range(len(output_sentences)):
        code_loc = []
        urls_loc=[]
        for c in range(len(code_positions)):
            if code_positions[c]>i:
                code_loc.append(c)
        for u in range(len(url_positions)):
            if url_positions[u]==i:
                urls_loc.append(u)
        knowledge_tuple = (code_loc,urls_loc)
        knowledge_dictionary.append(knowledge_tuple)
    # knowledge_dictionary stores the information to track the related code snippets and urls for 
    # each sentence in corresponding position. The 1st postition of tuple stores the indexes of code snippets
    # the second position of tuple stores the indexes of urls
    return final_output, code_snippets, urls, knowledge_dictionary


In [14]:
cleaned_text = cleaned_combined_data[72]["accepted_answer"]
# cleaned_text = preprocess_text_sentence(cleaned_combined_data[72]["accepted_answer"])
answer_sentences, code, urls, map= custom_sent_tokenize(cleaned_text)

In [20]:
# print(answer_sentences)
for i in range(len(answer_sentences)): 
    print(str(i)+":"+answer_sentences[i])
    print(map[i])
    print('-------------------')

0:I've just put together what you may be looking for: http://www.graphdracula.net
([0], [0])
-------------------
1:It's JavaScript with directed graph layouting, SVG and you can even drag the nodes around.
([0], [])
-------------------
2:Still needs some tweaking, but is totally usable.
([0], [])
-------------------
3:You create nodes and edges easily with JavaScript code like this:
([0], [])
-------------------
4:I used the previously mentioned Raphael JS library (the graffle example) plus some code for a force based graph layout algorithm I found on the net (everything open source, MIT license).
([], [])
-------------------
5:If you have any remarks or need a certain feature, I may implement it, just ask!
([], [])
-------------------
6:You may want to have a look at other projects, too!
([], [])
-------------------
7:Below are two meta-comparisons:
([], [])
-------------------
8:SocialCompare has an extensive list of libraries, and the "Node / edge graph" line will filter for graph v

In [17]:
print(code[0])

var g = new Graph();
g.addEdge("strawberry", "cherry");
g.addEdge("cherry", "apple");
g.addEdge("id34", "cherry");



In [23]:
print(urls[38])
print(urls[39])

https://code.google.com/p/jsdot/
https://code.google.com/p/jsdot/source/list


In [145]:
# print(answer_sentences[10])

Here's a list of similar projects (some have been already mentioned here):


In [63]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

device = "cuda"

para_tokenizer = AutoTokenizer.from_pretrained("humarin/chatgpt_paraphraser_on_T5_base")

para_model = AutoModelForSeq2SeqLM.from_pretrained("humarin/chatgpt_paraphraser_on_T5_base").to(device)

def paraphrase(
    question,
    num_beams=5,
    num_beam_groups=5,
    num_return_sequences=5,
    repetition_penalty=10.0,
    diversity_penalty=3.0,
    no_repeat_ngram_size=2,
    temperature=0.7,
    max_length=128
):
    input_ids = para_tokenizer(
        f'paraphrase: {question}',
        return_tensors="pt", padding="longest",
        max_length=max_length,
        truncation=True,
    ).input_ids
    
    input_ids = input_ids.to(device)

    outputs = para_model.generate(
        input_ids, temperature=temperature, repetition_penalty=repetition_penalty,
        num_return_sequences=num_return_sequences, no_repeat_ngram_size=no_repeat_ngram_size,
        num_beams=num_beams, num_beam_groups=num_beam_groups,
        max_length=max_length, diversity_penalty=diversity_penalty
    )

    res = para_tokenizer.batch_decode(outputs, skip_special_tokens=True)

    return res

In [64]:
model = SentenceTransformer('sentence-transformers/all-mpnet-base-v2')

In [65]:
embeddings_data = []

for data_item in tqdm(cleaned_combined_data):
    knowledge_pockets = {}
    raw_question = data_item['question']
    raw_accepted_answer = data_item['accepted_answer']

    # Compute embedding for the question
    question_embedding = model.encode(raw_question, convert_to_tensor=True)

    # Split the accepted answer into sentences
    answer_sentences, code_snippets, urls, knowledge_dictionary= custom_sent_tokenize(raw_accepted_answer)
    # cleaned_text = preprocess_text_sentence(raw_accepted_answer)
    # answer_sentences, code_snippets= custom_sent_tokenize(make_human_readable(cleaned_text))
    
    # Compute embeddings for each sentence in the answer
    # answer_embeddings = [model.encode(sentence, convert_to_tensor=True) for sentence in answer_sentences]
    knowledge_pockets['code_snippets'] = code_snippets
    knowledge_pockets['urls'] = urls
    knowledge_pockets['knowledge_dictionary'] = knowledge_dictionary
    # Store original text and embeddings in a single dictionary
    embeddings_data.append({
        'raw_question': raw_question,
        'question_embedding': question_embedding,
        'raw_accepted_answer': raw_accepted_answer,
        # 'answer_embeddings': answer_embeddings,
        'knowledge':knowledge_pockets
    })


100%|██████████| 14843/14843 [48:33<00:00,  5.09it/s] 


In [66]:
# len(embeddings_data[0]['answer_embeddings'])

1

In [67]:
import pickle
with open('CODE_POST_OVERALL-EMBEDDINGS_DATA.pkl', 'wb') as f:
    pickle.dump(embeddings_data, f)

In [5]:
import pickle
with open('CODE_POST_OVERALL-EMBEDDINGS_DATA.pkl', 'rb') as f:
    OVO_data = pickle.load(f)


In [69]:
# tag_list[1]

['java', 'oop', 'php', 'theory']

In [70]:
# if len(OVO_data) == len(tag_list):
#     for data_item, tag in zip(OVO_data, tag_list):
#         data_item['tag'] = tag
# else:
#     print("The lengths of OVO_data and tags_list do not match!")

# with open('OVERALL-EMBEDDINGS_DATA_V2.pkl', 'wb') as f:
#     pickle.dump(OVO_data, f)

In [6]:
OVO_data[0]

{'raw_question': 'Java lib or app to convert CSV to XML file?',
 'question_embedding': tensor([-5.9233e-02, -4.2513e-02, -3.7562e-02, -3.2367e-02,  1.4165e-02,
          3.5061e-02, -2.6379e-02,  5.0334e-02,  8.0433e-02,  5.9160e-02,
          1.6705e-03,  6.1885e-02,  3.4248e-02, -5.4983e-02, -2.4129e-02,
          3.6963e-02,  2.8554e-03,  3.9143e-02,  4.6939e-02,  2.0223e-02,
         -2.7318e-02,  1.3451e-02,  5.9952e-02,  1.8928e-02,  2.4985e-02,
         -2.6746e-02, -1.8312e-02, -1.0097e-02,  2.6363e-02, -6.5171e-02,
         -2.0231e-02, -1.9504e-02, -2.3118e-02,  1.8161e-02,  1.0242e-06,
         -1.0018e-02, -2.6278e-02,  7.9744e-03,  3.7028e-02,  3.2506e-02,
         -6.3649e-02, -5.4743e-02, -7.4604e-03,  3.2295e-02,  1.8438e-02,
          5.0273e-03, -4.7141e-03, -6.5190e-02,  5.5027e-02,  3.0269e-05,
         -1.5928e-03, -7.6427e-02,  2.8121e-02,  2.0102e-02,  4.5514e-02,
         -4.8995e-02, -1.1081e-02,  1.0915e-01, -1.7379e-02,  1.7555e-02,
          1.7683e-02, -3.9

In [ ]:
for single_item in tqdm(OVO_data):
    raw_accepted_answer = single_item['raw_accepted_answer']
    
    # Preprocess the accepted answer and tokenize it into sentences
    answer_sentences, code_snippets, urls, knowledge_dictionary= custom_sent_tokenize(raw_accepted_answer)
    
    # cleaned_text = preprocess_text_sentence(raw_accepted_answer)
    # answer_sentences, code= custom_sent_tokenize(make_human_readable(cleaned_text))
    
    # Add the computed answer_sentences to the dictionary
    single_item['answer_sentences'] = answer_sentences

with open('CODE_POST_OVERALL-EMBEDDINGS_DATA_V3.pkl', 'wb') as f:
    pickle.dump(OVO_data, f)

100%|██████████| 3428217/3428217 [10:11<00:00, 5604.28it/s]


In [75]:
# with open('CODE_POST_OVERALL-EMBEDDINGS_DATA_V2.pkl', 'rb') as f:
#     OVO_data_V2 = pickle.load(f)

In [9]:
OVO_data[0]

{'raw_question': 'Java lib or app to convert CSV to XML file?',
 'question_embedding': tensor([-5.9233e-02, -4.2513e-02, -3.7562e-02, -3.2367e-02,  1.4165e-02,
          3.5061e-02, -2.6379e-02,  5.0334e-02,  8.0433e-02,  5.9160e-02,
          1.6705e-03,  6.1885e-02,  3.4248e-02, -5.4983e-02, -2.4129e-02,
          3.6963e-02,  2.8554e-03,  3.9143e-02,  4.6939e-02,  2.0223e-02,
         -2.7318e-02,  1.3451e-02,  5.9952e-02,  1.8928e-02,  2.4985e-02,
         -2.6746e-02, -1.8312e-02, -1.0097e-02,  2.6363e-02, -6.5171e-02,
         -2.0231e-02, -1.9504e-02, -2.3118e-02,  1.8161e-02,  1.0242e-06,
         -1.0018e-02, -2.6278e-02,  7.9744e-03,  3.7028e-02,  3.2506e-02,
         -6.3649e-02, -5.4743e-02, -7.4604e-03,  3.2295e-02,  1.8438e-02,
          5.0273e-03, -4.7141e-03, -6.5190e-02,  5.5027e-02,  3.0269e-05,
         -1.5928e-03, -7.6427e-02,  2.8121e-02,  2.0102e-02,  4.5514e-02,
         -4.8995e-02, -1.1081e-02,  1.0915e-01, -1.7379e-02,  1.7555e-02,
          1.7683e-02, -3.9